In [ ]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
from config import api_key
from pandas.io.json import json_normalize  

## Get Scopus Author Information using Multiple Scopus Author IDs

In [ ]:
# File to Load
radiation_oncology = "radiation_oncology.csv"

# Read the CSV file and store into Pandas DataFrame with the column Scopus Author ID as a string
radiation_oncology_df = pd.read_csv(radiation_oncology, encoding="utf-8")

#Change the column names to lower case with underscore for spaces
radiation_oncology_df.columns =  radiation_oncology_df.columns.str.strip().str.lower().str.replace(" ", "_").str.replace("(","").str.replace(")","")
radiation_oncology_df.head()

In [ ]:
#List the column names in the dataframe
radiation_oncology_df.columns

In [ ]:
#Check the data type in the dataframe columns called scopus_author_id and scopus_search
# radiation_oncology_df.scopus_author_id.dtype
# radiation_oncology_df.scopus_search.dtype

In [ ]:
#Save the column called scopus_author_id to a list called Author_ID_List
author_ID_List = radiation_oncology_df['scopus_author_id'].tolist()
print(author_ID_List)

In [ ]:
#Clean the Author_ID_List to remove nan
cleaned_Author_ID_List = [x for x in author_ID_List if str(x) != 'nan']
print(cleaned_Author_ID_List)

In [ ]:
#Add the necessary syntax to the cleaned_Author_ID_List
scopus_Mulitple_AuthorID_Query = []
for x in cleaned_Author_ID_List:
    authorID_string = "".join(("AU-ID(", x,")"))
    scopus_Mulitple_AuthorID_Query.append(authorID_string)
    
print(scopus_Mulitple_AuthorID_Query)

In [ ]:
# scopus_Search_List = radiation_oncology_df['scopus_search'].tolist()
# print(scopus_Search_List)

In [ ]:
# cleaned_Scopus_Search_List = [x for x in scopus_Search_List if str(x) != 'nan']
# print(cleaned_Scopus_Search_List)

In [ ]:
#https://dev.elsevier.com/guides/ScopusSearchViews.htm
#https://stackoverflow.com/questions/53558837/python-loop-to-pull-api-data-for-iterating-urls
#https://stackoverflow.com/questions/36410800/python-3-parse-json-from-multiple-api-requests-into-a-list-and-output-to-a-fil
#https://www.pluralsight.com/guides/web-scraping-with-request-python

multiple_author_list = []
multiple_author_dict = {}
# outfilepath = "multiple_author_json.json"
# keys = ["0", "1", "2", "3", "4", "5", "6", "7", "8", "9", "10", "11", "12"]

def get_scopus_articles(scopus_Mulitple_AuthorID_Query):
    
    for authorid in scopus_Mulitple_AuthorID_Query:
        url = "http://api.elsevier.com/content/search/scopus?"
        fieldList = ["dc:identifier", "eid", "dc:title","prism:aggregationType", "subtype", "citedby-count",
                     "prism:publicationName","prism:volume","prism:issueIdentifier", "prism:pageRange", 
                     "prism:coverDate", "prism:doi","pubmed-id", "authid", "authname"]
                    
        headers = {
             "X-ELS-APIKey": api_key,
             'Accept':'application/json'
        }
        parameters = {
            "query": authorid,
            "field": ",".join(fieldList),
            "date": "2002-2003"
        }
        
        #Make the API request 
        single_author_response = requests.get(url, headers=headers, params=parameters)
        #print(single_author_response.url)
        #print(single_author_response.status_code)
        
              
        #Append each single_author_dict response to multiple_author_list to create a list of dictionaries
        single_author_dict = single_author_response.json()
        #print(type(single_author_dict)) 
        #print(single_author_dict)
        multiple_author_list.append(single_author_dict.copy())
    
    return multiple_author_list
       
get_scopus_articles(scopus_Mulitple_AuthorID_Query)



In [ ]:
#https://stackoverflow.com/questions/48177934/flatten-or-unpack-list-of-nested-dicts-in-dataframe
#https://stackoverflow.com/questions/50161070/convert-list-of-dicts-of-dict-into-dataframe
#https://stackoverflow.com/questions/43984865/python-having-trouble-returning-a-pandas-data-frame-from-a-user-defined-functio
#https://stackoverflow.com/questions/37668291/flatten-double-nested-json

def make_scopus_articles_df(multiple_author_list):
        #final_list = json_normalize(multiple_author_list, meta=["search-results"], record_path=["search-results", "entry"])
    scopus_articles_df = pd.DataFrame.from_dict(json_normalize(multiple_author_list, meta=["search-results"], record_path=["search-results", "entry"]),orient="columns")
    
    return scopus_articles_df

scopus_articles_df = make_scopus_articles_df(multiple_author_list)
make_scopus_articles_df(multiple_author_list)

In [ ]:
#https://stackoverflow.com/questions/29325458/dictionary-column-in-pandas-dataframe/29330853
def fix_dataframe(scopus_articles_df):
    remove_searchresults_nest = pd.concat([scopus_articles_df.drop(['search-results'], axis=1), scopus_articles_df['search-results'].apply(pd.Series)], axis=1)
    remove_opensearchQuery_nest = pd.concat([remove_searchresults_nest.drop(['opensearch:Query'], axis=1), remove_searchresults_nest['opensearch:Query'].apply(pd.Series)], axis=1)
    remove_opensearchQuery_nest = pd.concat([remove_searchresults_nest.drop(['opensearch:Query'], axis=1), remove_searchresults_nest['opensearch:Query'].apply(pd.Series)], axis=1)
    return remove_opensearchQuery_nest

remove_opensearchQuery_nest = fix_dataframe(scopus_articles_df)
fix_dataframe(scopus_articles_df)

In [ ]:
#remove_opensearchQuery_nest.to_csv (r'C:\Users\keg827\Documents\10. WorkStuff_KEG\scopusAPIrequests\export_dataframe.csv', index = None, header=True)

In [ ]:
#https://stackoverflow.com/questions/48637219/iterate-over-a-dictionary-of-list-of-dictionaries-in-pandas-dataframe
authorids = []
authornames = []


for item in remove_opensearchQuery_nest["author"][1]:
    #print(item.keys())
    #print(item.values())
    #print(item["authid"])
    authorids.append(item["authid"])
    authornames.append(item["authname"])
print(authornames)
print(authorids)  


In [ ]:
#new_df = remove_opensearchQuery_nest.fillna("0")
new_df = remove_opensearchQuery_nest

# new_df.loc[new_df['author'].isnull(), 'author'] = {}
for row in new_df.loc[new_df.author.isnull(), 'author'].index:
    new_df.at[row, 'author'] = []
    
new_df

In [ ]:
new_df.reindex(columns=[*new_df.columns.tolist(), 'author_id', 'author_name'])

In [ ]:
for row in new_df.loc[new_df.author_id.isnull(), 'author_id'].index:
    new_df.at[row, 'author_id'] = []
    
new_df

In [460]:
authorids = []
authornames = []
#string = new_df["author"]


def get_author_ids():
    authorids = []
    authornames = []
    for column in new_df['author']:
        #print(column)
        if column == []:
            continue
        else:
            for item in column:
                #print(item)
        #       print(item["authid"])
        #       print(column)
                authorids.append(item["authid"])
                print(authorids)
                #authornames.append(item["authname"])
            new_df["author_id"]= authorids 
        authorids=[]
        authornames = []
    
get_author_ids()

new_df.head()

#remove_opensearchQuery_nest.apply(lambda x: func(x['author']),axis=1)
# if column == "none":
#             continue
#         else:

['56018970700']
['56018970700', '6701662630']
['56018970700', '6701662630', '7404024068']
['56018970700', '6701662630', '7404024068', '6701854664']
['56018970700', '6701662630', '7404024068', '6701854664', '7102541014']
['56018970700', '6701662630', '7404024068', '6701854664', '7102541014', '7005140598']
['56018970700', '6701662630', '7404024068', '6701854664', '7102541014', '7005140598', '7102955789']
['56018970700', '6701662630', '7404024068', '6701854664', '7102541014', '7005140598', '7102955789', '6701449622']


ValueError: Length of values does not match length of index

In [459]:
authorids = []
authornames = []
#test = new_df["author"]
# for item in test.iteritems():
#     print(item.key)
# #     #print(item.values())
# #     #print(item["authid"])
#     authorids.append(item["authid"])
#     authornames.append(item["authname"])
# print(authornames)
# print(authorids)  


myList = new_df["author"][1]

# Enumerate ages
for i, age in enumerate(d['authid'] for d in myList): 
    print(authid)

    

NameError: name 'authid' is not defined

In [ ]:

# new_df['authorID_list'] = new_df['author'].apply(lambda x: x["authid"])
# new_df

new_df['authorIDs'] = 
